In [2]:
import numpy as np
import config
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import scipy
import utils
import time

plt_flag = True

target_aa_bit = np.array([0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1], dtype=np.uint8)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

def decode_and_check(phase_diff, preamble_idx, compare_num=32):
    if preamble_idx+8*config.sample_pre_symbol+compare_num*config.sample_pre_symbol > len(phase_diff):
        return False
    pd_seg = phase_diff[preamble_idx+8*config.sample_pre_symbol: preamble_idx+8*config.sample_pre_symbol+compare_num*config.sample_pre_symbol]
    phase_diff_bit = pd_seg.reshape(-1, config.sample_pre_symbol)
    vote = np.sum(phase_diff_bit, axis=1)
    bits = np.zeros(compare_num, dtype=np.uint8)
    bits[np.where(vote>0)[0]] = 1
    if (bits==target_aa_bit[:compare_num]).all():
        return True
    else:
        return False

def cal_phase_diff(signal):
    return signal[:, 0:-1].real * signal[:, 1:].imag - signal[:, 1:].real * signal[:, 0:-1].imag

# For low-frequency, the corresponding mask is -1, and 1 for high-frequency
def get_preamble_mask(extf):
    mask = torch.ones(extf * 8 * config.sample_pre_symbol, dtype=torch.float32)
    for i in range(0, 8, 2):
        mask[i*extf*config.sample_pre_symbol: (i+1)*extf*config.sample_pre_symbol] = -1
    return mask.view((1, 1, -1)).contiguous().to(device)

def cal_corrlation(phase_diff, mask):
    if isinstance(phase_diff, np.ndarray):
        phase_diff_tensor = torch.from_numpy(phase_diff).to(device)
        phase_diff_tensor = phase_diff_tensor.unsqueeze(1).type(torch.float32).to(device)
        out = F.conv1d(phase_diff_tensor, mask).squeeze(1)
        out = out.cpu().numpy()
    else:
        phase_diff_tensor = phase_diff.to(device)
        phase_diff_tensor = phase_diff_tensor.unsqueeze(1).type(torch.float32).to(device)
        out = F.conv1d(phase_diff_tensor, mask).squeeze(1)
        out = out.cpu()
    return out

# For phase difference, the correspongding sample point in the 
def peak_clustering_preamble_detection(corrlation, extf, cluster_size=3):
    max_diff = extf * config.sample_pre_symbol * 8
    peak_cluster_list = []
    peak_cluster_center_list = []
    for i in range(corrlation.shape[0]):
        peaks, properties = scipy.signal.find_peaks(corrlation[i, :], prominence=0, height=0)
        promin = properties["prominences"]
        promin_thres = np.percentile(promin, 80)
        height = properties["peak_heights"]
        height_thres = np.percentile(height, 80)
        peaks = peaks[np.where((promin>promin_thres)&(height>height_thres))]
        if len(peaks) > 0:
            peak_clusters = [[peaks[0]]]
            peak_cluster_center = [peaks[0]] # should be the peak that with maximum value
            for j in range(1, len(peaks)):
                # The peak can be clustered into the last cluster
                if peaks[j] - peak_cluster_center[-1] < max_diff:
                    # This peak is higher than the original center
                    # We consider it as the new center
                    if corrlation[i, peaks[j]] > corrlation[i, peak_cluster_center[-1]]:
                        peak_cluster_center[-1] = peaks[j]
                    # Otherwise, we juse append the peak into the cluster
                    peak_clusters[-1].append(peaks[j])
                # This peak is far way from the former cluster
                # we juse take it as a new cluster 
                else:
                    # If last cluster is too small, we can consider it as a noise
                    # Another exceptation is that the center is on one-side
                    # (cluster_size > 1 and (peak_cluster_center[-1] == peak_clusters[-1][0] or \
                    #     peak_cluster_center[-1] == peak_clusters[-1][-1]))
                    if len(peak_clusters[-1]) < cluster_size:
                        peak_clusters.pop()
                        peak_cluster_center.pop()
                    peak_clusters.append([peaks[j]])
                    peak_cluster_center.append(peaks[j])
        else:
            peak_clusters = []
            peak_cluster_center = [-1]
        
        # Check wether the last cluster is noise or not
        # (len(peak_clusters[-1]) < cluster_size or \
        #     (cluster_size > 1 and (peak_cluster_center[-1] == peak_clusters[-1][0] or \
        #     peak_cluster_center[-1] == peak_clusters[-1][-1]))):
        if len(peak_clusters) > 0 and len(peak_clusters[-1]) < cluster_size:
            peak_clusters.pop()
            peak_cluster_center.pop()
        peak_cluster_list.append(peak_clusters)
        peak_cluster_center_list.append(peak_cluster_center)
    
    return peak_cluster_list, peak_cluster_center_list


def preamble_add_noise(raw_signal, noise_signal, extf, peak_centers, snr):
    signal_len = raw_signal.shape[1]
    if noise_signal.shape[0] < raw_signal.shape[0]:
        noise_signal_tmp = np.zeros((raw_signal.shape[0], raw_signal.shape[1]), dtype=np.complex64)
        for i in range(raw_signal.shape[0]):
            noise_signal_tmp[i, :] = noise_signal[i % noise_signal.shape[0], (i // noise_signal.shape[0]) * raw_signal.shape[1]: (i // noise_signal.shape[0] + 1) * raw_signal.shape[1]]
        noise_signal = noise_signal_tmp
    else:
        noise_signal = noise_signal[: raw_signal.shape[0], : raw_signal.shape[1]]

    # Then we calculate the power of the preamble signal
    segment_len = 8 * config.sample_pre_symbol * extf
    real_signal_power = np.zeros(raw_signal.shape[0], dtype=np.float32)
    imag_signal_power = np.zeros(raw_signal.shape[0], dtype=np.float32)
    total_signal_len = np.zeros(raw_signal.shape[0], dtype=np.float32)
    for i in range(raw_signal.shape[0]):
        for center in peak_centers[i]:
            real_signal_power[i] += np.sum(np.real(raw_signal[i, center:center+segment_len])**2)
            imag_signal_power[i] += np.sum(np.imag(raw_signal[i, center:center+segment_len])**2)
        total_signal_len[i] = segment_len * len(peak_centers[i])
    real_signal_power /= total_signal_len
    imag_signal_power /= total_signal_len
   
    real_signal_power = real_signal_power.reshape(-1, 1)
    imag_signal_power = imag_signal_power.reshape(-1, 1)
    # Get the power of noise
    
    real_noise = np.real(noise_signal)
    imag_noise = np.imag(noise_signal)
    real_noise_power = (np.sum(real_noise ** 2, 1)).reshape(-1, 1) / signal_len
    imag_noise_power = (np.sum(imag_noise ** 2, 1)).reshape(-1, 1) / signal_len
    # for i in range(raw_signal.shape[0]):
    #     print(i, real_signal_power[i], imag_signal_power[i], "||", real_noise_power[i], imag_noise_power[i])
    real_signal_variance = (np.power(10., (snr/10))) * real_noise_power
    imag_signal_variance = (np.power(10., (snr/10))) * imag_noise_power
    print("preamble add", np.sqrt(real_signal_variance / real_signal_power)[0], np.sqrt(imag_signal_variance / imag_signal_power)[0])
    print(real_signal_power[0], imag_signal_power[0])
    print(real_noise_power[0], imag_noise_power[0])
    real_signal = np.sqrt(real_signal_variance / real_signal_power) * np.real(raw_signal) + real_noise
    imag_signal = np.sqrt(imag_signal_variance / imag_signal_power) * np.imag(raw_signal) + imag_noise
    signal_with_noise = np.zeros((raw_signal.shape[0], raw_signal.shape[1]), dtype=np.complex64)
    signal_with_noise.real = real_signal
    signal_with_noise.imag = imag_signal
    # return signal_with_noise
    b, a = scipy.signal.butter(8, 2e6 / config.sample_rate, "lowpass")
    filtered_signal = scipy.signal.filtfilt(b, a, signal_with_noise, 1)
    return filtered_signal

def preamble_detection_stft(signal_ori, extf, window_size_sym=1):
    # We copy the memory since the following operations will change the data
    signal = torch.tensor(signal_ori).to(device)
    # signal = torch.tensor(signal_seg)
    window_size = window_size_sym * config.sample_pre_symbol
    # Calcualte the STFT of the signal
    # If the extension is smaller than the window size, we pad 0s after each segment
    if extf > window_size_sym:
        signal_window = signal.unfold(1, window_size, 1)
    else:
        signal_window = signal.unfold(1, extf * config.sample_pre_symbol, 1)
        left_pads_num = (window_size_sym * config.sample_pre_symbol - extf * config.sample_pre_symbol) // 2
        right_pads_num = window_size_sym * config.sample_pre_symbol - extf * config.sample_pre_symbol - left_pads_num
        left_pad = torch.zeros((signal_window.shape[0], signal_window.shape[1], left_pads_num), dtype=torch.complex64).to(device)
        right_pad = torch.zeros((signal_window.shape[0], signal_window.shape[1], right_pads_num), dtype=torch.complex64).to(device)
        signal_window = torch.cat((left_pad, signal_window, right_pad), dim=2)
    # Remove the DC value
    signal_mean = torch.mean(signal_window, dim=2).unsqueeze(2)
    signal_window = signal_window - signal_mean
    hanning_windows = torch.hann_window(window_size).to(device)
    signal_window = signal_window * hanning_windows

    # Start STFT, this tensor should be [batch, freq, time]
    stft_res = torch.fft.fft(signal_window, dim=2).transpose(1, 2).contiguous()
    # Combine the high and low frequency
    comb_res = torch.zeros(stft_res.shape[0], 2, stft_res.shape[2], dtype=torch.float32).to(device)
    freq_len = stft_res.shape[1] // 2
    if stft_res.shape[1] % 2 == 0:
        comb_res[:, 0, :] = torch.sum(torch.abs(stft_res[:, :freq_len, :]), dim=1)
        comb_res[:, 1, :] = torch.sum(torch.abs(stft_res[:, freq_len:, :]), dim=1)
    else:
        comb_res[:, 0, :] = torch.sum(torch.abs(stft_res[:, :freq_len, :]), dim=1)
        comb_res[:, 1, :] = torch.sum(torch.abs(stft_res[:, (freq_len+1):, :]), dim=1)
    comb_res = comb_res.unsqueeze(1)
    # Then we start to calculate the corrlaton between the current time-frequency plot and the target
    stft_mask = torch.zeros(2, config.sample_pre_symbol * extf * 8).to(device)
    for i in range(8):
        if i % 2 == 0:
            stft_mask[0, i*extf*config.sample_pre_symbol: (i+1)*extf*config.sample_pre_symbol] = -1
            stft_mask[1, i*extf*config.sample_pre_symbol: (i+1)*extf*config.sample_pre_symbol] = 1
        else:
            stft_mask[0, i*extf*config.sample_pre_symbol: (i+1)*extf*config.sample_pre_symbol] = 1
            stft_mask[1, i*extf*config.sample_pre_symbol: (i+1)*extf*config.sample_pre_symbol] = -1
    stft_mask = stft_mask.unsqueeze(0).unsqueeze(0)
    corrlation = F.conv2d(comb_res, stft_mask).squeeze(1).squeeze(1).cpu().numpy()
    
    test_idx = 0
    pc, pc_center = peak_clustering_preamble_detection(corrlation, extf, cluster_size=3)
    for i in range(len(pc_center)):
        for j in range(len(pc_center[i])):
            pc_center[i][j] += window_size // 2
    return pc_center

def preamble_detection_phase_diff(signal_ori, extf, mask=None, cluster_size=3):
    if mask is None:
        mask = get_preamble_mask(extf)
    phase_diff = cal_phase_diff(signal_ori)
    corrlation = cal_corrlation(phase_diff, mask)
    pc, pc_center = peak_clustering_preamble_detection(corrlation, extf, cluster_size=3)
    # np.savetxt("./export_data/preamble_detect/corrlation.csv", corrlation[0, :], delimiter=",")
    if plt_flag:
        phase_cum = np.cumsum(phase_diff, axis=1)
        plot_peaks(corrlation, phase_cum, pc, pc_center, extf, prefix="phase_diff_")
    # Each phase diff at index i is calculated by the i and i+1 sample point
    # if phase_diff>0, the i+1 sample point is 1, and if phase_diff<0, the i+1 sample point is 0
    # sample pint: 0   1   2   3   4   5
    # phase diff:    0   1   2   3   4
    # for i in range(len(pc_center)):
    #     for j in range(len(pc_center[i])):
    #         pc_center[i][j] += 1
    return pc_center, corrlation

plt_flag = False
config.sample_rate = 20e6
config.sample_pre_symbol = 20

if __name__ == "__main__":
    ori_sample_pre_symbol = config.sample_pre_symbol
    ori_sample_rate = config.sample_rate
    batch_size = 8
    for extf in [1, 2, 4, 8, 16, 32, 64]:
        # [1, 2, 4, 8, 16, 32, 64]
        # white_noise = np.vstack((np.load("./raw_data/white_noise_1_chan=8.npz")["arr_0"], np.load("./raw_data/white_noise_1_chan=8.npz")["arr_0"]))
        data = np.load("../raw_data/single_preamble_0_chan=8_extf=" + str(extf) + ".npz")
        noise_signal = np.load("../raw_data/white_noise.npy")
        
        raw_signal = data["arr_0"]
        raw_signal = raw_signal[:, :100000]
        raw_signal = utils.filter(raw_signal)
        print(raw_signal.shape)
        noise_signal = noise_signal[:, :raw_signal.shape[1]]
        print("ext=", extf)
        preamble_phase_mask = get_preamble_mask(extf)
        # Get the ground truth
        phase_diff = cal_phase_diff(raw_signal)
        pc_center, corrlation = preamble_detection_phase_diff(raw_signal, extf, preamble_phase_mask, cluster_size=3)
        # stft_pc_2_tmp = preamble_detection_stft(raw_signal, extf, 2)
        # stft_pc = np.array(stft_pc_2_tmp).reshape(-1)
        phase = utils.get_phase_cum(raw_signal)
        print("phase_shape",phase.shape)
        centers = np.zeros(len(pc_center), dtype=np.int32)
        centers[:] = -1
        preamble_tops = np.array([ind for ind in range(0, config.sample_pre_symbol * extf * 8, config.sample_pre_symbol * extf)], dtype=np.int32)

        valid_idx = []
        for i in range(len(pc_center)):
            corr_val = np.array([corrlation[i, p] for p in pc_center[i]])
            if extf == 1:
                idx_sort = np.argsort(-corr_val)
                for j in idx_sort:
                    if decode_and_check(phase_diff[i, :], pc_center[i][j]):
                        centers[i] = pc_center[i][j]
                        break
                if centers[i] == -1:
                    print(i, "th has no preamble")
                else:
                    valid_idx.append(i)
            else:
                max_idx = np.argmax(corr_val)
                centers[i] = pc_center[i][max_idx]
                valid_idx.append(i)
        valid_idx = np.array(valid_idx)
        raw_signal = raw_signal[valid_idx]
        centers = centers[valid_idx]
        raw_signal = np.vstack([raw_signal for _ in range(8)])
        centers = np.concatenate([centers for _ in range(8)])
        noise_signal = noise_signal[:raw_signal.shape[0]]
        centers = centers[:, np.newaxis]
        batch_num = raw_signal.shape[0] // batch_size
        preamble_points = np.arange(0, 8*extf*config.sample_pre_symbol, extf*config.sample_pre_symbol)
        # stft_pc_1_tmp = preamble_detection_stft(raw_signal[:8 ], extf, 1)
        # print(np.mean(centers))
        # for i in [36]:
        #     plt.figure(figsize=(12, 6))
        #     plt.subplot(2,1,1)
        #     plt.plot(phase[i, :])
        #     plt.subplot(2,1,2)
        #     print(centers[i, 0])
        #     plt.plot(corrlation[i, :])
        # print(np.mean(centers))
        
        train_precent = 0.8
        # start with 1packets: stft add window_size // 2
        # start with packets: stft no add with window_size // 2
        f  = open("../output/packet_detection/awgn_packet_detection_{}.csv".format(extf), 'w')
        detail_f = open("../output/packet_detection/awgn_packet_detection_{}_detail.csv".format(extf), 'w')
        for db in range(-30, 11, 1):
            print(db, end=",")
            # signal = preamble_add_noise(raw_signal, noise_signal[:, :raw_signal.shape[1]], extf, centers, db).copy()
            signal = utils.preamble_awgn(raw_signal, db, centers, extf, noise_signal).copy()
            # if extf == 8:
            #     signal2 = utils.preamble_awgn(raw_signal, db, centers, extf, noise_signal).copy()
            #     signal = preamble_add_noise(raw_signal, noise_signal[:, :raw_signal.shape[1]], extf, centers, db).copy()
            #     plt.figure()
            #     plt.plot(signal[0, :].real)
            #     plt.plot(signal[0, :].imag)
            #     plt.figure()
            #     plt.plot(signal2[0, :].real)
            #     plt.plot(signal2[0, :].imag)
            #     plt.figure()
            #     plt.plot(noise_signal[0, :].real)
            #     plt.plot(noise_signal[0, :].imag)
                
            # else:
            #     signal = preamble_add_noise(raw_signal, noise_signal[:, :raw_signal.shape[1]], extf, centers, db).copy()
            times = [time.time()]
            stft_pc_1 = []
            for i in range(batch_num):
                stft_pc_1_tmp = preamble_detection_stft(signal[batch_size*i:batch_size*(i+1)], extf, 1)
                stft_pc_1 = stft_pc_1 + stft_pc_1_tmp
            times.append(time.time())
            stft_pc_2 = []
            for i in range(batch_num):
                stft_pc_2_tmp = preamble_detection_stft(signal[batch_size*i:batch_size*(i+1)], extf, 2)
                stft_pc_2 = stft_pc_2 + stft_pc_2_tmp
            times.append(time.time())
            phase_diff_pc = []
            for i in range(batch_num):
                phase_diff_pc_tmp, _ = preamble_detection_phase_diff(signal[batch_size*i:batch_size*(i+1)], extf, preamble_phase_mask, cluster_size=3)
                phase_diff_pc = phase_diff_pc + phase_diff_pc_tmp
            times.append(time.time())
            pcs = [stft_pc_1, stft_pc_2, phase_diff_pc]
            acc = np.zeros(3)
            detail_detect_num = np.zeros([raw_signal.shape[0], 3], dtype=np.int32)
            detect_num = np.zeros(3)
            for i in range(raw_signal.shape[0]):
                for j in range(3):
                    # print(j, i)
                    if len(pcs[j][i]) > 0:
                        idx = np.array(pcs[j][i])
                        diff = np.abs(idx - centers[i])
                        min_diff = np.min(diff)
                        if min_diff < (extf * config.sample_pre_symbol / 2):
                            acc[j] += min_diff
                            detect_num[j] += 1
                            detail_detect_num[i, j] = 1
            times = np.array(times)
            times[1: ] = times[1: ] - times[: -1]
            try:
                acc = acc / detect_num
            except BaseException:
                pass
            detect_prob = detect_num / raw_signal.shape[0]
            line = "extf={},db={}, acc=[{}, {}, {}], detect_prob = [{}, {}, {}]".format(extf, db, acc[0], acc[1], acc[2], detect_prob[0], detect_prob[1], detect_prob[2])
            print(line)
            line = "{},{},{},{},{},{},{},{}\n".format(extf, db, acc[0], acc[1], acc[2], detect_prob[0], detect_prob[1], detect_prob[2])
            f.write(line)
            train_num = int(train_precent * raw_signal.shape[0])
            train_detr = np.mean(detail_detect_num[:train_num, :], axis=0)
            test_detr = np.mean(detail_detect_num[train_num:, :], axis=0)
            detail_line = "{}, {}, {}, {}, {}, {}, {}, {}\n".format(extf, db, train_detr[0], train_detr[1], train_detr[2], test_detr[0], test_detr[1], test_detr[2])
            print(detail_line)
            detail_f.write(detail_line)
            # print(stft_pc_1[0], centers[0, 0])
            for i in range(raw_signal.shape[0]):
                    # print(j, i)
                if len(pcs[0][i]) > 0:
                    idx = np.array(pcs[0][i])
                    diff = np.abs(idx - centers[i])
                    min_diff = np.min(diff)
                    min_diff_idx = np.argmin(diff)
                    diff_sign = idx - centers[i]
                    # if min_diff < (extf * config.sample_pre_symbol ):
                    #     print(diff_sign[min_diff_idx])
        f.flush()
        f.close()
        detail_f.flush()
        detail_f.close()

(128, 2400)
ext= 1
phase_shape (128, 2399)
36 th has no preamble
-30,extf=1,db=-30, acc=[3.789473684210526, 4.444444444444445, 4.2727272727272725], detect_prob = [0.018700787401574805, 0.008858267716535433, 0.010826771653543307]
1, -30, 0.019704433497536946, 0.009852216748768473, 0.011083743842364532, 0.014705882352941176, 0.004901960784313725, 0.00980392156862745

-29,extf=1,db=-29, acc=[4.0, 4.444444444444445, 4.2727272727272725], detect_prob = [0.01968503937007874, 0.008858267716535433, 0.010826771653543307]
1, -29, 0.020935960591133004, 0.009852216748768473, 0.011083743842364532, 0.014705882352941176, 0.004901960784313725, 0.00980392156862745

-28,extf=1,db=-28, acc=[3.8947368421052633, 4.090909090909091, 4.454545454545454], detect_prob = [0.018700787401574805, 0.010826771653543307, 0.010826771653543307]
1, -28, 0.019704433497536946, 0.012315270935960592, 0.012315270935960592, 0.014705882352941176, 0.004901960784313725, 0.004901960784313725

-27,extf=1,db=-27, acc=[3.75, 4.0, 4.461

In [3]:
import numpy as np
import config
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import scipy
import utils
import time

plt_flag = True

target_aa_bit = np.array([0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1], dtype=np.uint8)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

def decode_and_check(phase_diff, preamble_idx, compare_num=32):
    if preamble_idx+8*config.sample_pre_symbol+compare_num*config.sample_pre_symbol > len(phase_diff):
        return False
    pd_seg = phase_diff[preamble_idx+8*config.sample_pre_symbol: preamble_idx+8*config.sample_pre_symbol+compare_num*config.sample_pre_symbol]
    phase_diff_bit = pd_seg.reshape(-1, config.sample_pre_symbol)
    vote = np.sum(phase_diff_bit, axis=1)
    bits = np.zeros(compare_num, dtype=np.uint8)
    bits[np.where(vote>0)[0]] = 1
    if (bits==target_aa_bit[:compare_num]).all():
        return True
    else:
        return False

def cal_phase_diff(signal):
    return signal[:, 0:-1].real * signal[:, 1:].imag - signal[:, 1:].real * signal[:, 0:-1].imag

# For low-frequency, the corresponding mask is -1, and 1 for high-frequency
def get_preamble_mask(extf):
    mask = torch.ones(extf * 8 * config.sample_pre_symbol, dtype=torch.float32)
    for i in range(0, 8, 2):
        mask[i*extf*config.sample_pre_symbol: (i+1)*extf*config.sample_pre_symbol] = -1
    return mask.view((1, 1, -1)).contiguous().to(device)

def cal_corrlation(phase_diff, mask):
    if isinstance(phase_diff, np.ndarray):
        phase_diff_tensor = torch.from_numpy(phase_diff).to(device)
        phase_diff_tensor = phase_diff_tensor.unsqueeze(1).type(torch.float32).to(device)
        out = F.conv1d(phase_diff_tensor, mask).squeeze(1)
        out = out.cpu().numpy()
    else:
        phase_diff_tensor = phase_diff.to(device)
        phase_diff_tensor = phase_diff_tensor.unsqueeze(1).type(torch.float32).to(device)
        out = F.conv1d(phase_diff_tensor, mask).squeeze(1)
        out = out.cpu()
    return out

# For phase difference, the correspongding sample point in the 
def peak_clustering_preamble_detection(corrlation, extf, cluster_size=3):
    max_diff = extf * config.sample_pre_symbol * 8
    peak_cluster_list = []
    peak_cluster_center_list = []
    for i in range(corrlation.shape[0]):
        peaks, properties = scipy.signal.find_peaks(corrlation[i, :], prominence=0, height=0)
        promin = properties["prominences"]
        promin_thres = np.percentile(promin, 80)
        height = properties["peak_heights"]
        height_thres = np.percentile(height, 80)
        peaks = peaks[np.where((promin>promin_thres)&(height>height_thres))]
        if len(peaks) > 0:
            peak_clusters = [[peaks[0]]]
            peak_cluster_center = [peaks[0]] # should be the peak that with maximum value
            for j in range(1, len(peaks)):
                # The peak can be clustered into the last cluster
                if peaks[j] - peak_cluster_center[-1] < max_diff:
                    # This peak is higher than the original center
                    # We consider it as the new center
                    if corrlation[i, peaks[j]] > corrlation[i, peak_cluster_center[-1]]:
                        peak_cluster_center[-1] = peaks[j]
                    # Otherwise, we juse append the peak into the cluster
                    peak_clusters[-1].append(peaks[j])
                # This peak is far way from the former cluster
                # we juse take it as a new cluster 
                else:
                    # If last cluster is too small, we can consider it as a noise
                    # Another exceptation is that the center is on one-side
                    # (cluster_size > 1 and (peak_cluster_center[-1] == peak_clusters[-1][0] or \
                    #     peak_cluster_center[-1] == peak_clusters[-1][-1]))
                    if len(peak_clusters[-1]) < cluster_size:
                        peak_clusters.pop()
                        peak_cluster_center.pop()
                    peak_clusters.append([peaks[j]])
                    peak_cluster_center.append(peaks[j])
        else:
            peak_clusters = []
            peak_cluster_center = [-1]
        
        # Check wether the last cluster is noise or not
        # (len(peak_clusters[-1]) < cluster_size or \
        #     (cluster_size > 1 and (peak_cluster_center[-1] == peak_clusters[-1][0] or \
        #     peak_cluster_center[-1] == peak_clusters[-1][-1]))):
        if len(peak_clusters) > 0 and len(peak_clusters[-1]) < cluster_size:
            peak_clusters.pop()
            peak_cluster_center.pop()
        peak_cluster_list.append(peak_clusters)
        peak_cluster_center_list.append(peak_cluster_center)
    
    return peak_cluster_list, peak_cluster_center_list


def preamble_add_noise(raw_signal, noise_signal, extf, peak_centers, snr):
    signal_len = raw_signal.shape[1]
    if noise_signal.shape[0] < raw_signal.shape[0]:
        noise_signal_tmp = np.zeros((raw_signal.shape[0], raw_signal.shape[1]), dtype=np.complex64)
        for i in range(raw_signal.shape[0]):
            noise_signal_tmp[i, :] = noise_signal[i % noise_signal.shape[0], (i // noise_signal.shape[0]) * raw_signal.shape[1]: (i // noise_signal.shape[0] + 1) * raw_signal.shape[1]]
        noise_signal = noise_signal_tmp
    else:
        noise_signal = noise_signal[: raw_signal.shape[0], : raw_signal.shape[1]]

    # Then we calculate the power of the preamble signal
    segment_len = 8 * config.sample_pre_symbol * extf
    real_signal_power = np.zeros(raw_signal.shape[0], dtype=np.float32)
    imag_signal_power = np.zeros(raw_signal.shape[0], dtype=np.float32)
    total_signal_len = np.zeros(raw_signal.shape[0], dtype=np.float32)
    for i in range(raw_signal.shape[0]):
        for center in peak_centers[i]:
            real_signal_power[i] += np.sum(np.real(raw_signal[i, center:center+segment_len])**2)
            imag_signal_power[i] += np.sum(np.imag(raw_signal[i, center:center+segment_len])**2)
        total_signal_len[i] = segment_len * len(peak_centers[i])
    real_signal_power /= total_signal_len
    imag_signal_power /= total_signal_len
   
    real_signal_power = real_signal_power.reshape(-1, 1)
    imag_signal_power = imag_signal_power.reshape(-1, 1)
    # Get the power of noise
    
    real_noise = np.real(noise_signal)
    imag_noise = np.imag(noise_signal)
    real_noise_power = (np.sum(real_noise ** 2, 1)).reshape(-1, 1) / signal_len
    imag_noise_power = (np.sum(imag_noise ** 2, 1)).reshape(-1, 1) / signal_len
    # for i in range(raw_signal.shape[0]):
    #     print(i, real_signal_power[i], imag_signal_power[i], "||", real_noise_power[i], imag_noise_power[i])
    real_signal_variance = (np.power(10., (snr/10))) * real_noise_power
    imag_signal_variance = (np.power(10., (snr/10))) * imag_noise_power
    print("preamble add", np.sqrt(real_signal_variance / real_signal_power)[0], np.sqrt(imag_signal_variance / imag_signal_power)[0])
    print(real_signal_power[0], imag_signal_power[0])
    print(real_noise_power[0], imag_noise_power[0])
    real_signal = np.sqrt(real_signal_variance / real_signal_power) * np.real(raw_signal) + real_noise
    imag_signal = np.sqrt(imag_signal_variance / imag_signal_power) * np.imag(raw_signal) + imag_noise
    signal_with_noise = np.zeros((raw_signal.shape[0], raw_signal.shape[1]), dtype=np.complex64)
    signal_with_noise.real = real_signal
    signal_with_noise.imag = imag_signal
    # return signal_with_noise
    b, a = scipy.signal.butter(8, 2e6 / config.sample_rate, "lowpass")
    filtered_signal = scipy.signal.filtfilt(b, a, signal_with_noise, 1)
    return filtered_signal

def preamble_detection_stft(signal_ori, extf, window_size_sym=1):
    # We copy the memory since the following operations will change the data
    signal = torch.tensor(signal_ori).to(device)
    # signal = torch.tensor(signal_seg)
    window_size = window_size_sym * config.sample_pre_symbol
    # Calcualte the STFT of the signal
    # If the extension is smaller than the window size, we pad 0s after each segment
    if extf > window_size_sym:
        signal_window = signal.unfold(1, window_size, 1)
    else:
        signal_window = signal.unfold(1, extf * config.sample_pre_symbol, 1)
        left_pads_num = (window_size_sym * config.sample_pre_symbol - extf * config.sample_pre_symbol) // 2
        right_pads_num = window_size_sym * config.sample_pre_symbol - extf * config.sample_pre_symbol - left_pads_num
        left_pad = torch.zeros((signal_window.shape[0], signal_window.shape[1], left_pads_num), dtype=torch.complex64).to(device)
        right_pad = torch.zeros((signal_window.shape[0], signal_window.shape[1], right_pads_num), dtype=torch.complex64).to(device)
        signal_window = torch.cat((left_pad, signal_window, right_pad), dim=2)
    # Remove the DC value
    signal_mean = torch.mean(signal_window, dim=2).unsqueeze(2)
    signal_window = signal_window - signal_mean
    hanning_windows = torch.hann_window(window_size).to(device)
    signal_window = signal_window * hanning_windows

    # Start STFT, this tensor should be [batch, freq, time]
    stft_res = torch.fft.fft(signal_window, dim=2).transpose(1, 2).contiguous()
    # Combine the high and low frequency
    comb_res = torch.zeros(stft_res.shape[0], 2, stft_res.shape[2], dtype=torch.float32).to(device)
    freq_len = stft_res.shape[1] // 2
    if stft_res.shape[1] % 2 == 0:
        comb_res[:, 0, :] = torch.sum(torch.abs(stft_res[:, :freq_len, :]), dim=1)
        comb_res[:, 1, :] = torch.sum(torch.abs(stft_res[:, freq_len:, :]), dim=1)
    else:
        comb_res[:, 0, :] = torch.sum(torch.abs(stft_res[:, :freq_len, :]), dim=1)
        comb_res[:, 1, :] = torch.sum(torch.abs(stft_res[:, (freq_len+1):, :]), dim=1)
    comb_res = comb_res.unsqueeze(1)
    # Then we start to calculate the corrlaton between the current time-frequency plot and the target
    stft_mask = torch.zeros(2, config.sample_pre_symbol * extf * 8).to(device)
    for i in range(8):
        if i % 2 == 0:
            stft_mask[0, i*extf*config.sample_pre_symbol: (i+1)*extf*config.sample_pre_symbol] = -1
            stft_mask[1, i*extf*config.sample_pre_symbol: (i+1)*extf*config.sample_pre_symbol] = 1
        else:
            stft_mask[0, i*extf*config.sample_pre_symbol: (i+1)*extf*config.sample_pre_symbol] = 1
            stft_mask[1, i*extf*config.sample_pre_symbol: (i+1)*extf*config.sample_pre_symbol] = -1
    stft_mask = stft_mask.unsqueeze(0).unsqueeze(0)
    corrlation = F.conv2d(comb_res, stft_mask).squeeze(1).squeeze(1).cpu().numpy()
    
    test_idx = 0
    pc, pc_center = peak_clustering_preamble_detection(corrlation, extf, cluster_size=3)
    for i in range(len(pc_center)):
        for j in range(len(pc_center[i])):
            pc_center[i][j] += window_size // 2
    return pc_center

def preamble_detection_phase_diff(signal_ori, extf, mask=None, cluster_size=3):
    if mask is None:
        mask = get_preamble_mask(extf)
    phase_diff = cal_phase_diff(signal_ori)
    corrlation = cal_corrlation(phase_diff, mask)
    pc, pc_center = peak_clustering_preamble_detection(corrlation, extf, cluster_size=3)
    # np.savetxt("./export_data/preamble_detect/corrlation.csv", corrlation[0, :], delimiter=",")
    if plt_flag:
        phase_cum = np.cumsum(phase_diff, axis=1)
        plot_peaks(corrlation, phase_cum, pc, pc_center, extf, prefix="phase_diff_")
    # Each phase diff at index i is calculated by the i and i+1 sample point
    # if phase_diff>0, the i+1 sample point is 1, and if phase_diff<0, the i+1 sample point is 0
    # sample pint: 0   1   2   3   4   5
    # phase diff:    0   1   2   3   4
    # for i in range(len(pc_center)):
    #     for j in range(len(pc_center[i])):
    #         pc_center[i][j] += 1
    return pc_center, corrlation

plt_flag = False
config.sample_rate = 20e6
config.sample_pre_symbol = 20

if __name__ == "__main__":
    ori_sample_pre_symbol = config.sample_pre_symbol
    ori_sample_rate = config.sample_rate
    batch_size = 8
    for extf in [1, 2, 4, 8, 16, 32, 64]:
        # [2, 4, 8, 16, 32, 64, 128]
        # white_noise = np.vstack((np.load("./raw_data/white_noise_1_chan=8.npz")["arr_0"], np.load("./raw_data/white_noise_1_chan=8.npz")["arr_0"]))
        data = np.load("../raw_data/single_preamble_0_chan=8_extf=" + str(extf) + ".npz")
        noise_signal = np.load("../raw_data/white_noise.npy")
        
        raw_signal = data["arr_0"]
        raw_signal = raw_signal[:, :100000]
        raw_signal = utils.filter(raw_signal)
        print(raw_signal.shape)
        noise_signal = noise_signal[:, :raw_signal.shape[1]]
        print("ext=", extf)
        preamble_phase_mask = get_preamble_mask(extf)
        # Get the ground truth
        phase_diff = cal_phase_diff(raw_signal)
        pc_center, corrlation = preamble_detection_phase_diff(raw_signal, extf, preamble_phase_mask, cluster_size=3)
        # stft_pc_2_tmp = preamble_detection_stft(raw_signal, extf, 2)
        # stft_pc = np.array(stft_pc_2_tmp).reshape(-1)
        phase = utils.get_phase_cum(raw_signal)
        print("phase_shape",phase.shape)
        centers = np.zeros(len(pc_center), dtype=np.int32)
        centers[:] = -1
        preamble_tops = np.array([ind for ind in range(0, config.sample_pre_symbol * extf * 8, config.sample_pre_symbol * extf)], dtype=np.int32)

        valid_idx = []
        for i in range(len(pc_center)):
            corr_val = np.array([corrlation[i, p] for p in pc_center[i]])
            if extf == 1:
                idx_sort = np.argsort(-corr_val)
                for j in idx_sort:
                    if decode_and_check(phase_diff[i, :], pc_center[i][j]):
                        centers[i] = pc_center[i][j]
                        break
                if centers[i] == -1:
                    print(i, "th has no preamble")
                else:
                    valid_idx.append(i)
            else:
                max_idx = np.argmax(corr_val)
                centers[i] = pc_center[i][max_idx]
                valid_idx.append(i)
        valid_idx = np.array(valid_idx)
        raw_signal = raw_signal[valid_idx]
        centers = centers[valid_idx]
        raw_signal = np.vstack([raw_signal for _ in range(8)])
        centers = np.concatenate([centers for _ in range(8)])
        noise_signal = noise_signal[:raw_signal.shape[0]]
        centers = centers[:, np.newaxis]
        batch_num = raw_signal.shape[0] // batch_size
        preamble_points = np.arange(0, 8*extf*config.sample_pre_symbol, extf*config.sample_pre_symbol)
        # stft_pc_1_tmp = preamble_detection_stft(raw_signal[:8 ], extf, 1)
        # print(np.mean(centers))
        # for i in [36]:
        #     plt.figure(figsize=(12, 6))
        #     plt.subplot(2,1,1)
        #     plt.plot(phase[i, :])
        #     plt.subplot(2,1,2)
        #     print(centers[i, 0])
        #     plt.plot(corrlation[i, :])
        # print(np.mean(centers))
        
        train_precent = 0.8
        # start with 1packets: stft add window_size // 2
        # start with packets: stft no add with window_size // 2
        f  = open("../output/packet_detection/filter_packet_detection_{}.csv".format(extf), 'w')
        detail_f = open("../output/packet_detection/filter_packet_detection_{}_detail.csv".format(extf), 'w')
        for db in range(-30, 11, 1):
            print(db, end=",")
            signal = preamble_add_noise(raw_signal, noise_signal[:, :raw_signal.shape[1]], extf, centers, db).copy()
            # if extf == 8:
            #     signal2 = utils.preamble_awgn(raw_signal, db, centers, extf, noise_signal).copy()
            #     signal = preamble_add_noise(raw_signal, noise_signal[:, :raw_signal.shape[1]], extf, centers, db).copy()
            #     plt.figure()
            #     plt.plot(signal[0, :].real)
            #     plt.plot(signal[0, :].imag)
            #     plt.figure()
            #     plt.plot(signal2[0, :].real)
            #     plt.plot(signal2[0, :].imag)
            #     plt.figure()
            #     plt.plot(noise_signal[0, :].real)
            #     plt.plot(noise_signal[0, :].imag)
                
            # else:
            #     signal = preamble_add_noise(raw_signal, noise_signal[:, :raw_signal.shape[1]], extf, centers, db).copy()
            times = [time.time()]
            stft_pc_1 = []
            for i in range(batch_num):
                stft_pc_1_tmp = preamble_detection_stft(signal[batch_size*i:batch_size*(i+1)], extf, 1)
                stft_pc_1 = stft_pc_1 + stft_pc_1_tmp
            times.append(time.time())
            stft_pc_2 = []
            for i in range(batch_num):
                stft_pc_2_tmp = preamble_detection_stft(signal[batch_size*i:batch_size*(i+1)], extf, 2)
                stft_pc_2 = stft_pc_2 + stft_pc_2_tmp
            times.append(time.time())
            phase_diff_pc = []
            for i in range(batch_num):
                phase_diff_pc_tmp, _ = preamble_detection_phase_diff(signal[batch_size*i:batch_size*(i+1)], extf, preamble_phase_mask, cluster_size=3)
                phase_diff_pc = phase_diff_pc + phase_diff_pc_tmp
            times.append(time.time())
            pcs = [stft_pc_1, stft_pc_2, phase_diff_pc]
            acc = np.zeros(3)
            detail_detect_num = np.zeros([raw_signal.shape[0], 3], dtype=np.int32)
            detect_num = np.zeros(3)
            for i in range(raw_signal.shape[0]):
                for j in range(3):
                    # print(j, i)
                    if len(pcs[j][i]) > 0:
                        idx = np.array(pcs[j][i])
                        diff = np.abs(idx - centers[i])
                        min_diff = np.min(diff)
                        if min_diff < (extf * config.sample_pre_symbol / 2):
                            acc[j] += min_diff
                            detect_num[j] += 1
                            detail_detect_num[i, j] = 1
            times = np.array(times)
            times[1: ] = times[1: ] - times[: -1]
            try:
                acc = acc / detect_num
            except BaseException:
                pass
            detect_prob = detect_num / raw_signal.shape[0]
            line = "extf={},db={}, acc=[{}, {}, {}], detect_prob = [{}, {}, {}]".format(extf, db, acc[0], acc[1], acc[2], detect_prob[0], detect_prob[1], detect_prob[2])
            print(line)
            line = "{},{},{},{},{},{},{},{}\n".format(extf, db, acc[0], acc[1], acc[2], detect_prob[0], detect_prob[1], detect_prob[2])
            f.write(line)
            train_num = int(train_precent * raw_signal.shape[0])
            train_detr = np.mean(detail_detect_num[:train_num, :], axis=0)
            test_detr = np.mean(detail_detect_num[train_num:, :], axis=0)
            detail_line = "{}, {}, {}, {}, {}, {}, {}, {}\n".format(extf, db, train_detr[0], train_detr[1], train_detr[2], test_detr[0], test_detr[1], test_detr[2])
            print(detail_line)
            detail_f.write(detail_line)
            # print(stft_pc_1[0], centers[0, 0])
            for i in range(raw_signal.shape[0]):
                    # print(j, i)
                if len(pcs[0][i]) > 0:
                    idx = np.array(pcs[0][i])
                    diff = np.abs(idx - centers[i])
                    min_diff = np.min(diff)
                    min_diff_idx = np.argmin(diff)
                    diff_sign = idx - centers[i]
                    # if min_diff < (extf * config.sample_pre_symbol ):
                    #     print(diff_sign[min_diff_idx])
        f.flush()
        f.close()
        detail_f.flush()
        detail_f.close()

(128, 2400)
ext= 1
phase_shape (128, 2399)
36 th has no preamble
-30,preamble add [0.56624407] [1.2461234]
[0.00311645] [0.00066509]
[0.99923307] [1.0327672]


extf=1,db=-30, acc=[3.9545454545454546, 3.875, 5.25], detect_prob = [0.021653543307086614, 0.007874015748031496, 0.011811023622047244]
1, -30, 0.023399014778325122, 0.008620689655172414, 0.013546798029556651, 0.014705882352941176, 0.004901960784313725, 0.004901960784313725

-29,preamble add [0.6353363] [1.3981735]
[0.00311645] [0.00066509]
[0.99923307] [1.0327672]
extf=1,db=-29, acc=[3.8181818181818183, 3.875, 5.25], detect_prob = [0.021653543307086614, 0.007874015748031496, 0.011811023622047244]
1, -29, 0.023399014778325122, 0.008620689655172414, 0.013546798029556651, 0.014705882352941176, 0.004901960784313725, 0.004901960784313725

-28,preamble add [0.71285903] [1.5687765]
[0.00311645] [0.00066509]
[0.99923307] [1.0327672]
extf=1,db=-28, acc=[3.95, 3.875, 5.083333333333333], detect_prob = [0.01968503937007874, 0.007874015748031496, 0.011811023622047244]
1, -28, 0.019704433497536946, 0.009852216748768473, 0.012315270935960592, 0.0196078431372549, 0.0, 0.00980392156862745

-27,preamble

In [5]:
import numpy as np
errors = []
extfs = [2,4, 8, 16, 32, 64]
for extf in extfs:
    data = np.loadtxt("/liymdata/liym/BLong/output/packet_detection/filter_packet_detection_{}.csv".format(extf), delimiter=",")[15:31, 2:5]
    error = data / (extf * 20)
    error = np.mean(error, axis=0)
    errors.append(error)
    
print(errors)
errors = np.array(errors)
e1 = errors[:, 2] - errors[:, 0]
e2 = errors[:, 2] - errors[:, 1]
print("to stft 1us", np.min(e1), np.max(e1))
print("to stft 2us", np.min(e2), np.max(e2))
np.savetxt("/liymdata/liym/BLong/output/packet_detection/filter_packet_detection_error.csv", np.hstack([np.array(extfs, dtype=np.int32).reshape(-1, 1), errors]), delimiter=",")

[array([0.20151713, 0.17902975, 0.08099117]), array([0.16040568, 0.12004041, 0.05413069]), array([0.12098477, 0.08553528, 0.03925946]), array([0.09980151, 0.06046716, 0.0240021 ]), array([0.07610248, 0.03715686, 0.01268858]), array([0.05356483, 0.02223827, 0.00665157])]
to stft 1us -0.12052596278427494 -0.046913259334766616
to stft 2us -0.09803857896004078 -0.01558670323895301
